Librerias que se usaran para el analisis

In [2]:
from datetime import datetime
import pandas as pd
import numpy as np

ETL_Archivo (extract, transform and load)

In [5]:
#se extrae el dataset 'online sales data' de forma local luego head() proporciona uba visiob de los primeros 5 registros
data=pd.read_csv(r'C:/Users/flord/Downloads/onlinesales/Online Sales Data.csv')
data.head()

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue,Region,Payment Method
0,10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card
1,10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal
2,10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card
3,10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card
4,10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal


Busqueda de elementos nulos

In [51]:
data.isnull().sum()

Transaction ID      0
Date                0
Product Category    0
Product Name        0
Units Sold          0
Unit Price          0
Total Revenue       0
Region              0
Payment Method      0
total               0
dtype: int64

Revision si existe datos duplicados

In [52]:
data.duplicated().sum()

0

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    240 non-null    int64  
 1   Date              240 non-null    object 
 2   Product Category  240 non-null    object 
 3   Product Name      240 non-null    object 
 4   Units Sold        240 non-null    int64  
 5   Unit Price        240 non-null    float64
 6   Total Revenue     240 non-null    float64
 7   Region            240 non-null    object 
 8   Payment Method    240 non-null    object 
 9   total             240 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 18.9+ KB


Transformaciones que se hacen a cada columna del dataset

In [54]:

data['Product Category'] = data['Product Category'].astype('str')
data['Product Name'] = data['Product Name'].astype('str')
data['Region'] = data['Region'].astype('str')
data['Payment Method'] = data['Payment Method'].astype('str')

Conexion de python a PostgreSQL para cargar los datos 

In [62]:
import psycopg2 

conn = psycopg2.connect(
    host='localhost',
    dbname='postgres',
    user='postgres',
    password='1234',
    port='5432'
)

# Crear un cursor para ejecutar comandos en la base de datos
cur = conn.cursor()
# Crear la tabla ventas si no existe
cur.execute("""
    CREATE TABLE IF NOT EXISTS ventas (
        transact_id INT,
        date VARCHAR(50),
        product_category VARCHAR(150),
        product_name VARCHAR(150),
        units_sold INT,
        unit_price NUMERIC(10, 2),
        total_revenue NUMERIC(10, 2),
        region VARCHAR(150),
        payment_method VARCHAR(150)
    );
""")
conn.commit()

# Insertar datos en la tabla ventas 
for i, row in data.iterrows():
    cur.execute("""
        INSERT INTO ventas (transact_id, date, product_category, product_name, units_sold, unit_price, total_revenue, region, payment_method)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (transact_id) DO NOTHING;
    """, (
        row['Transaction ID'],
        row['Date'],
        row['Product Category'],
        row['Product Name'],
        row['Units Sold'],
        row['Unit Price'],
        row['Total Revenue'],
        row['Region'],
        row['Payment Method']
    ))

# Confirmar los cambios en la base de datos
conn.commit()

# Cerrar el cursor y la conexión
cur.close()
conn.close()
